# Pulling search results from nextdoor
Inspired by the following [github repo](https://github.com/chanwooh/Nextdoor-Script/blob/master/nextdoor.py) and a [branch](https://github.com/towhidabsar/Nextdoor-Script/blob/master/nextdoor.py).

Modified to account for string searches and updated css tags.

The following code will open nextdoor.com, enter credentials and search for a string using the stringsearch variable. 

Requires chrome driver executable:
https://chromedriver.chromium.org/downloads

In [2]:
%%time
# Python Version 3.8.0

from selenium import webdriver
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.keys import Keys

import pandas as pd
import progressbar
import time

from lxml import html

# Set up driver options
capa = DesiredCapabilities.CHROME
capa["pageLoadStrategy"] = "none"

# Set up driver
driver = webdriver.Chrome(desired_capabilities=capa, executable_path=r"chromedriver.exe")
driver.get("https://nextdoor.com/login/")

time.sleep(10)

# Log In
#find the username and password text boxes
username = driver.find_element("id","id_email")
password = driver.find_element("id","id_password")

#send the username and password and click sign in
username.send_keys("gmichael@ucsd.edu")
password.send_keys("Family93536!")
driver.find_element("xpath", '//button[@id="signin_button"]').click()
time.sleep(10) # if not scrolling in time, make this number larger

print('logged in')

# Click the pop up, if one appears
try:
    driver.find_element("xpath", "//button[@class='channels-bulk-join-close-button']").click()
except:
    pass

#change the string to the keyword you'd like to search for
stringsearch = "Assault"

#find the textbox where you search for a value and enter the keyword
search = driver.find_element("xpath", value='//input[@id="search-input-field"]')
search.send_keys(stringsearch)
search.send_keys(Keys.ENTER)
time.sleep(5)
print('searched for', stringsearch)
#when you click search, results bring up matches for many categories. choose the "posts" tab to find matching posts
driver.find_element("xpath", value='.//a[@data-testid="tab-posts"]').click()

#due to the high scrape time, set up a progress bar since this will take a while
#progress bar https://pypi.org/project/progressbar/
#https://www.geeksforgeeks.org/progress-bars-in-python/
widgets = [' [',
         progressbar.Timer(format= 'elapsed time: %(elapsed)s'),
         '] ',
           progressbar.Bar('*'),' (',
           progressbar.ETA(), ') ',
          ]

#set number of scrolls
num_scrolls = 10

#initialize the progressbar
bar = progressbar.ProgressBar(maxval = num_scrolls - 1).start()

# Use Selenium to scroll 'range' number of times
# Change the second number in 'range(x, y)' to determine how many times you want it to scroll down.
for i in range(0, num_scrolls):

    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(4)
    bar.update(i)

time.sleep(1)
print('finished scrolling ' + str(num_scrolls) + ' times.')

# Scrape the page source returned from Chrome driver for posts
html_source = driver.page_source
readable_html = html_source.encode('utf-8')
tree = html.fromstring(readable_html)
postNodes = tree.xpath('//div[@class="css-15luflj"]')

# Iterate over each post node that has an author to get data in an organized fashion

author_path = '(.//div[@class="css-77j1dk"]//span/text())[1]'
location_date_path = '(.//div[@class="css-77j1dk"]//span/text())[2]'
post_content_path = './/div[@class="css-1i0jvqo"]/span/text()'
link_path = './/a[@class="uUGmI2_t css-rbdvzr"]'

posts = [(post.xpath(author_path),
          post.xpath(location_date_path),
          post.xpath(post_content_path),
          post.xpath('.//a[@class="uUGmI2_t css-rbdvzr"]')[0].attrib['href'],
          post) for post in postNodes if post.xpath(author_path) != []]


driver.quit()

<timed exec>:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


logged in


searched for Assault


finished scrolling 10 times.
CPU times: total: 62.5 ms
Wall time: 1min 14s


In [37]:
from datetime import datetime

Save the result to a csv

In [38]:
df = pd.DataFrame(posts, columns = ['Name', 'Location/Date', 'Content', 'Link', 'xpath'])
df['DateExtracted']= datetime.today().strftime('%Y-%m-%d')
df.to_csv(<YOURPATH> + stringsearch + '.csv')
df

,Name,Location/Date,Content,Link,xpath,DateExtracted
0,[San Diego Police Department Public Informatio...,[2 Jul],[A man is in police custody as two bystanders ...,https://nextdoor.com/p/GtT9Zzb2z6yz?view=detai...,[[[<Element div at 0x238021e2c20>]]],2022-11-21
1,[Jewels Mar],[Normal Heights Central · 25 Oct],[Saw this on the door and window of Dimille’s ...,https://nextdoor.com/p/b68LN26ypr3f?view=detai...,[[[<Element div at 0x238021e2ae0>]]],2022-11-21
2,[Heather Martinez],[San Carlos-Mission Gorge · 30 Oct],"[Good morning neighbors, last night around 7:3...",https://nextdoor.com/p/t6KDJZgTh3Xy/c/85201395...,[[[<Element div at 0x238021e2c20>]]],2022-11-21
3,[Ian Tolentino],[College · 27 Oct],"[Someone steals $400 from your shop, your cler...",https://nextdoor.com/p/JwbsdyzrKwtx/c/84857584...,[[[<Element div at 0x238021e2ae0>]]],2022-11-21
4,[Kay Blaine],[University Heights West · 25 Oct],[This man just spit on my face while walking m...,https://nextdoor.com/p/K9TpK8w2PNyq/c/84699715...,[[[<Element div at 0x238021e2c20>]]],2022-11-21
...,...,...,...,...,...,...
1268,[Lee Clark],[North Park-Morley Field · 26 Jul],"[Hello Everyone, , No one and no place is safe...",https://nextdoor.com/p/gbK_hCGFRcSL/c/80342549...,[[[<Element div at 0x238021e2ae0>]]],2022-11-21
1269,[Michelle S.],[Midtown · 28 Sep],"[#GrandTheftAuto #CarThief , …able to steal bi...",https://nextdoor.com/p/6x8_DqTyHZgc/c/83593241...,[[[<Element div at 0x238021e2c20>]]],2022-11-21
1270,[Lenny Burns],[Talmadge · 18 Oct],[Whoever was driving the white Chevy Silverado...,https://nextdoor.com/p/fLxQzG-7sSDt/c/84471486...,[[[<Element div at 0x238021e2ae0>]]],2022-11-21
1271,[Tom S.],[Heart of Linda Vista · 25 Jan],[UPDATE: I’ve been in contact with a couple of...,https://nextdoor.com/p/Zxh2NPMD5Lr6/c/72462112...,[[[<Element div at 0x238021e2c20>]]],2022-11-21
